In [218]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler

In [204]:
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [205]:
def From_pyparquet_to_pandascsv(load_path, save_path) :
    df = pq.read_table(load_path).to_pandas()
    df.to_csv(save_path, index = False, sep = ";" )
    return os.path.isfile(save_path)

#### New model approch Dexter news variables VBM

In [4]:
df_final_armand = spark.read.parquet("/Users/youssouf/Documents/simboxv2/df_final_v4.parquet")

df_final_armand.show(5)

+----------+-------------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------+
|date_appel|       msisdn|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|statut|
+----------+-------------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------+
|2021-04-11|2250707492509|           6|                3|           11|   1017|                 7|           13|          0|               0|           0|     0|                0|             0

##### Data VBM Dexter

In [5]:
df_vbm_fr = spark.read.option("header", True)\
                    .option("delimiter",";")\
                    .csv("/Users/youssouf/Downloads/vbm_avec_type_terminal_smartphoneflag/feature_vbm_fraudeur_20211008.csv")

df_vbm_fr.show(5)


df_vbm_normal = spark.read.option("header", True)\
                    .option("delimiter",";")\
                    .csv("/Users/youssouf/Downloads/vbm_avec_type_terminal_smartphoneflag/feature_vbm_fraudeur_non_20211008.csv")

df_vbm_normal.show(5)

+-------------+--------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-------------+--------------+
|       MSISDN|  DISTRIBUTEUR|STATUT_VOIX|STATUT_SMS|STATUT_FF|STATUT_SVA|STATUT_DATA|STATUT_OM|VALEUR|VOL_TOT_DATA|VOL_TOT_VOIX_OFFNET|VOL_TOT_VOIX_ONET|TYPE_TERMINAL|SMARTPHONE_FLG|
+-------------+--------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-------------+--------------+
|2250797351141|     SUITE COM|          1|         0|        0|         1|          0|        0| 854.0|         0.0|  31.06666666666666|              0.0|          GSM|             0|
|2250777519667|           SII|          0|         0|        0|         0|          0|        0|   0.0|         0.0|                0.0|              0.0|      INCONNU|             0|
|2250748795688|           TID|          0|         0|        0|         0|      

In [6]:
df_vbm = df_vbm_normal.union(df_vbm_fr)

df_vbm.show(5)

+-------------+-----------------+-----------+----------+---------+----------+-----------+---------+-------+-----------------+-------------------+-----------------+-------------+--------------+
|       MSISDN|     DISTRIBUTEUR|STATUT_VOIX|STATUT_SMS|STATUT_FF|STATUT_SVA|STATUT_DATA|STATUT_OM| VALEUR|     VOL_TOT_DATA|VOL_TOT_VOIX_OFFNET|VOL_TOT_VOIX_ONET|TYPE_TERMINAL|SMARTPHONE_FLG|
+-------------+-----------------+-----------+----------+---------+----------+-----------+---------+-------+-----------------+-------------------+-----------------+-------------+--------------+
|2250787780688|LEAN DISTRIBUTION|          1|         1|        0|         1|          0|        1|1570.71|              0.0|                0.0|             43.3|        OTHER|             0|
|2250789257579|          INCONNU|          0|         0|        0|         0|          0|        1|    0.0|              0.0|                0.0|              0.0|      INCONNU|             0|
|2250708206416|LEAN DISTRIBUTION|  

In [7]:
cols = [i.lower() for i in df_vbm.columns]
df_vbm = df_vbm.toDF(*(c for c in cols))

In [8]:
df_vbm.write.format("parquet").mode("overwrite").save("/Users/youssouf/Documents/simboxv2/vbm.parquet")


##### Joining Final and VBM

In [9]:
df_final_vbm = df_final_armand.join(df_vbm, ["msisdn"], "left")
df_final_vbm.show()


+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------+------------+-----------+----------+---------+----------+-----------+---------+-------+------------+-------------------+------------------+-------------+--------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|statut|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om| valeur|vol_tot_data|vol_tot_voix_offnet| vol_tot_voix_onet|type_terminal|smartphone_flg|
+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+--

In [10]:
df_final_vbm.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- date_appel: string (nullable = true)
 |-- nbre_sms_out: string (nullable = true)
 |-- nbre_dist_sms_out: string (nullable = true)
 |-- nbre_voix_out: string (nullable = true)
 |-- dur_out: string (nullable = true)
 |-- nbre_dist_voix_out: string (nullable = true)
 |-- nbre_cell_out: string (nullable = true)
 |-- nbre_sms_in: string (nullable = true)
 |-- nbre_dist_sms_in: string (nullable = true)
 |-- nbre_voix_in: string (nullable = true)
 |-- dur_in: string (nullable = true)
 |-- nbre_dist_voix_in: string (nullable = true)
 |-- nbre_sms_night: string (nullable = true)
 |-- nbre_dist_sms_night: string (nullable = true)
 |-- nbre_voix_night: string (nullable = true)
 |-- dur_night: string (nullable = true)
 |-- nbre_dist_voix_night: string (nullable = true)
 |-- statut: integer (nullable = true)
 |-- distributeur: string (nullable = true)
 |-- statut_voix: string (nullable = true)
 |-- statut_sms: string (nullable = true)
 |-- statut_ff: 

In [11]:
df_final_vbm.write.format("parquet").mode("overwrite").save("/Users/youssouf/Documents/simboxv2/df_final_vbm.parquet")



In [12]:
df_final_vbm.select("smartphone_flg","type_terminal").show(5)

+--------------+-------------+
|smartphone_flg|type_terminal|
+--------------+-------------+
|             0|      INCONNU|
|             0|        OTHER|
|             1|           4G|
|             1|           4G|
|             0|        OTHER|
+--------------+-------------+
only showing top 5 rows



#### Jrnée du 16

In [13]:
liste_fra_16 = pd.read_csv("/Users/youssouf/Documents/simboxv2/liste_fr_sb_16.csv")
liste_fra_16.head()

liste_fra_161 = liste_fra_16.msisdn.to_list()

In [101]:
liste_tmp = spark.read.option("delimiter",";").option("header", True).csv("/Users/youssouf/Documents/simboxv2/liste_fr_sb_16.csv")
liste_tmp.show(5)


+-------------+
|       msisdn|
+-------------+
|2250797114287|
|2250759715341|
|2250779057084|
|2250789027081|
|2250767279929|
+-------------+
only showing top 5 rows



In [103]:
dt_vba_dist_16.join(liste_tmp, ["msisdn"], "inner").filter(col("prediction") == 1.0).count()

254

In [14]:
dt_vba_dist_16 = spark.read.parquet("/Users/youssouf/Documents/simboxv2/prediction_dt_vbm_16_v4.parquet")

dt_vba_dist_16.show()

dt_vba_dist_16.groupby("prediction").count().show()

df_fra_predit = dt_vba_dist_16.filter(dt_vba_dist_16.msisdn.isin(liste_fra_161))

df_fra_predit.filter(col("prediction") == 1.0).select("msisdn").distinct().count()

+-------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-------------+--------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+-----------------+------------------+----------------+-----------------+--------------------+----------------+--------------------+----------+
|       msisdn|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|type_terminal|smartphone_flg|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|dis

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0| 348094|
|       1.0|7459279|
+----------+-------+



254

In [15]:
dt_vba_dist_162 = dt_vba_dist_16

dt_vba_dist_162 = dt_vba_dist_162.withColumn("probability_split", vector_to_array("probability"))
dt_vba_dist_162 = dt_vba_dist_162.select([col("probability_split")[i] for i in range(2)] + dt_vba_dist_162.columns)


dt_vba_dist_162.filter((dt_vba_dist_162.msisdn.isin(liste_fra_161)) & (col("prediction") == 1.0) )\
       .groupby("probability_split[1]").count().show()


+--------------------+-----+
|probability_split[1]|count|
+--------------------+-----+
|  0.9200561009817672|    1|
|  0.9238555147768979|  253|
+--------------------+-----+



In [16]:
dt_vba_dist_162 = dt_vba_dist_162.withColumn("pred", when((col("probability_split[1]") >= 0.92) & (col("probability_split[1]") < 0.93)  , 1).otherwise(0))


dt_vba_dist_162.filter((dt_vba_dist_162.msisdn.isin(liste_fra_161)) & (col("pred") == 1.0) ).select("msisdn").distinct().count()


254

In [17]:
dt_vba_dist_162.groupby("pred").count().show()


+----+-------+
|pred|  count|
+----+-------+
|   1|7451816|
|   0| 355557|
+----+-------+



In [45]:
dt_vba_dist_162.filter((dt_vba_dist_162.msisdn.isin(liste_fra_161)) & (col("pred") == 1.0) ).select("msisdn", "distributeur").show()



+-------------+------------+
|       msisdn|distributeur|
+-------------+------------+
|2250797114287|         SII|
|2250759715341|         SII|
|2250779057084|         SII|
|2250789027081|         SII|
|2250767279929|         SII|
|2250787076564|         SII|
|2250798700121|         SII|
|2250709007692|         SII|
|2250787727477|         SII|
|2250700061202|         SII|
|2250768857991|         SII|
|2250798944335|         SII|
|2250700028272|         SII|
|2250789360227|         SII|
|2250798946051|         SII|
|2250788033408|         SII|
|2250798965019|         SII|
|2250798844676|         SII|
|2250777079856|         SII|
|2250798748877|         SII|
+-------------+------------+
only showing top 20 rows



#### Data Transaction OM

In [18]:
df_om = spark.read.option("delimiter",";").option("header", True).csv("/Users/youssouf/Downloads/histo_trans_om_202109.csv")
df_om.show(5)


+------+----------+-------------+----------------+
| month|    msisdn|nbre_trans_om|montant_trans_om|
+------+----------+-------------+----------------+
|202109|0101145655|           61|        381145.0|
|202109|0101181859|           52|       1153600.0|
|202109|0101305149|            1|          2100.0|
|202109|0101505091|            5|          1025.0|
|202109|0101858593|            4|          4950.0|
+------+----------+-------------+----------------+
only showing top 5 rows



In [19]:
df_om.groupby("month").count().show()

+------+-------+
| month|  count|
+------+-------+
|202109|5689715|
+------+-------+



In [20]:
df_om.groupby("msisdn").count().orderBy(["count"] , ascending = True).show()

+----------+-----+
|    msisdn|count|
+----------+-----+
|0700013582|    1|
|0700023275|    1|
|0700041703|    1|
|0700041837|    1|
|0700046210|    1|
|0700047215|    1|
|0700050653|    1|
|0700051018|    1|
|0700053058|    1|
|0700104046|    1|
|0700104275|    1|
|0700106488|    1|
|0700106819|    1|
|0700144763|    1|
|0700147353|    1|
|0700174971|    1|
|0700183816|    1|
|0700187995|    1|
|0700199417|    1|
|0700217047|    1|
+----------+-----+
only showing top 20 rows



#### Jrnée du 27/09/2021

In [21]:
df_27 = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20210927.parquet")
df_27.show(5)

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+--------------+-----------+----------+---------+----------+-----------+---------+-------+------------+-------------------+-----------------+-----------------+----------------+--------------------+---------------+-------------------+-------------+------------+----------+------------+------------+-----------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|  distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om| valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|distr

In [22]:
df_27.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- date_appel: string (nullable = true)
 |-- nbre_sms_out: double (nullable = true)
 |-- nbre_dist_sms_out: double (nullable = true)
 |-- nbre_voix_out: double (nullable = true)
 |-- dur_out: double (nullable = true)
 |-- nbre_dist_voix_out: double (nullable = true)
 |-- nbre_cell_out: double (nullable = true)
 |-- nbre_sms_in: double (nullable = true)
 |-- nbre_dist_sms_in: double (nullable = true)
 |-- nbre_voix_in: double (nullable = true)
 |-- dur_in: double (nullable = true)
 |-- nbre_dist_voix_in: double (nullable = true)
 |-- nbre_sms_night: double (nullable = true)
 |-- nbre_dist_sms_night: double (nullable = true)
 |-- nbre_voix_night: double (nullable = true)
 |-- dur_night: double (nullable = true)
 |-- nbre_dist_voix_night: double (nullable = true)
 |-- distributeur: string (nullable = true)
 |-- statut_voix: double (nullable = true)
 |-- statut_sms: double (nullable = true)
 |-- statut_ff: double (nullable = true)
 |-- statut_sv

In [23]:
df_27.count()

176618

In [26]:
df_27.select("msisdn").write.format("parquet").mode("overwrite").save("/Users/youssouf/Documents/simboxv2/msisdn_fraudeur27.parquet")


In [29]:
From_pyparquet_to_pandascsv("/Users/youssouf/Documents/simboxv2/msisdn_fraudeur27.parquet", "/Users/youssouf/Documents/simboxv2/msisdn_fraudeur27.csv")





True

In [33]:
df_vbm_juillet = spark.read\
    .option("header", True)\
    .option("delimiter",";")\
    .csv("/Users/youssouf/Downloads/data_final/fraudeur_simbox_vbm_all_juill.csv")


In [35]:
df_vbm_juillet.show(5)

+-------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+
|       MSISDN|DISTRIBUTEUR|STATUT_VOIX|STATUT_SMS|STATUT_FF|STATUT_SVA|STATUT_DATA|STATUT_OM|VALEUR|VOL_TOT_DATA|VOL_TOT_VOIX_OFFNET|VOL_TOT_VOIX_ONET|
+-------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+
|2250798932246|     INCONNU|          0|         0|        0|         0|          0|        0|     0|           0|                  0|                0|
|2250797198120|     INCONNU|          0|         0|        0|         0|          1|        0|     0|       57355|                  0|                0|
|2250799564481|         AJS|          1|         1|        1|         1|          1|        1|  7089|       45360|                411|               72|
|2250797891567|         SII|          0|         0|        0|         0|          

In [41]:
df_vbm_juillet.groupby("DISTRIBUTEUR").count().orderBy(desc("count")).show()

+-----------------+-------+
|     DISTRIBUTEUR|  count|
+-----------------+-------+
|   MOSAIK TELECOM|2301562|
|              SII|2026098|
|              AJS|1718098|
|           AMANDA|1605098|
|         PRIMO MS|1326668|
|       ID TELECOM|1317408|
|LEAN DISTRIBUTION|1226204|
|        SUITE COM|1170745|
|              TID|1108878|
|              KTC| 987032|
|     DISTRICENTRE| 959195|
|      LA COQUILLE| 932723|
|             TKGS| 909538|
|   MAGIC BUSINESS| 785043|
|          INCONNU| 488412|
|          SODITEL| 373878|
+-----------------+-------+



In [44]:
df_vbm_juillet.groupby("DISTRIBUTEUR").count().orderBy(desc("count")).first()

Row(DISTRIBUTEUR='MOSAIK TELECOM', count=2301562)

In [37]:
df_vbm = spark.read\
    .option("header", True)\
    .option("delimiter",";")\
    .csv("/Users/youssouf/Downloads/fraudeur_simbox_vbm_all.csv")

In [40]:
df_vbm.groupby("DISTRIBUTEUR").count().orderBy(desc("count")).show()

+-----------------+-------+
|     DISTRIBUTEUR|  count|
+-----------------+-------+
|   MOSAIK TELECOM|2449727|
|              SII|2130848|
|              AJS|1723699|
|           AMANDA|1613341|
|         PRIMO MS|1322161|
|       ID TELECOM|1315433|
|LEAN DISTRIBUTION|1231843|
|        SUITE COM|1209589|
|              TID|1109446|
|              KTC|1026368|
|     DISTRICENTRE| 950670|
|      LA COQUILLE| 950138|
|             TKGS| 910187|
|   MAGIC BUSINESS| 801131|
|          INCONNU| 455285|
|          SODITEL| 371870|
+-----------------+-------+



#### Jrnée du 28

In [46]:
df_28 = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20210928.parquet")
df_28.show(5)

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+--------------+-----------+----------+---------+----------+-----------+---------+-------+------------+-------------------+-----------------+-----------------+----------------+--------------------+---------------+-------------------+-------------+------------+----------+------------+------------+-----------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|  distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om| valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|distr

In [47]:
df_28.count()

361472

In [49]:
df_28.select("msisdn").join(df_27, ["msisdn"], "inner").count()

252685

#### DF fraudeurs du 28/09/2021 FRA

In [60]:
df_fraudeur_FRA = spark.read.option("header", True).option("delimiter", ";").csv("/Users/youssouf/Downloads/simbox_fra_2809_0610.csv")
df_fraudeur_FRA.show(5)



+--------------+--------+
|Date_DETECTION|  MSISDN|
+--------------+--------+
|    28/09/2021|00004846|
|    28/09/2021|00048563|
|    28/09/2021|00058736|
|    28/09/2021|00058747|
|    28/09/2021|00058748|
+--------------+--------+
only showing top 5 rows



In [61]:
df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN")))
df_fraudeur_FRA.show(5)


+--------------+-------------+
|Date_DETECTION|       msisdn|
+--------------+-------------+
|    28/09/2021|2250700004846|
|    28/09/2021|2250700048563|
|    28/09/2021|2250700058736|
|    28/09/2021|2250700058747|
|    28/09/2021|2250700058748|
+--------------+-------------+
only showing top 5 rows



In [54]:
df_fraudeur_28_FRA = df_fraudeur_FRA.filter(col("Date_DETECTION") == "28/09/2021")
df_fraudeur_28_FRA.show(5)


+--------------+-------------+
|Date_DETECTION|       msisdn|
+--------------+-------------+
|    28/09/2021|2250700004846|
|    28/09/2021|2250700048563|
|    28/09/2021|2250700058736|
|    28/09/2021|2250700058747|
|    28/09/2021|2250700058748|
+--------------+-------------+
only showing top 5 rows



In [55]:
df_fraudeur_28_FRA.join(df_28, ["msisdn"], "inner").count()

0

#### Jrnée 29

In [57]:
df_29 = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20210929.parquet")
df_29.show()

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+--------------+-----------+----------+---------+----------+-----------+---------+-------+------------+-------------------+-----------------+-----------------+----------------+--------------------+---------------+-------------------+-------------+------------+----------+------------+------------+-----------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|  distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om| valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|distr

In [58]:
df_29.count()

170773

In [62]:
df_FRA_29 = df_fraudeur_FRA.filter(col("Date_DETECTION") == "29/09/2021")
df_FRA_29.show(5)

df_FRA_29.count()

+--------------+-------------+
|Date_DETECTION|       msisdn|
+--------------+-------------+
|    29/09/2021|2250700041842|
|    29/09/2021|2250700047788|
|    29/09/2021|2250700047792|
|    29/09/2021|2250700047888|
|    29/09/2021|2250700048119|
+--------------+-------------+
only showing top 5 rows



307

In [63]:
df_29.join(df_FRA_29, ["msisdn"], "inner").count()

0

In [64]:
df_29.select("msisdn", "prediction_filter", "prediction", "probability").show(5)

+-------------+-----------------+----------+------------+
|       msisdn|prediction_filter|prediction| probability|
+-------------+-----------------+----------+------------+
|2250707021090|                1|       1.0|[0.02, 0.98]|
|2250707036961|                1|       1.0|[0.02, 0.98]|
|2250707037586|                1|       1.0|[0.02, 0.98]|
|2250707043654|                1|       1.0|[0.02, 0.98]|
|2250707046210|                1|       1.0|[0.02, 0.98]|
+-------------+-----------------+----------+------------+
only showing top 5 rows



In [72]:
from datetime import datetime


stri = '9-15-18'
date_object = datetime.strptime(stri , '%m-%d-%y')
#date_object

date_object_i = datetime.datetime(date_object, '%m/%d/%y')
date_object_i
#date_object = datetime.strptime(stri , '%m/%d/%y')


AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [75]:
from datetime import datetime


In [81]:

d = datetime.strptime("20130125", '%Y%m%d')
print(datetime.strftime(d, "%d/%m/%y"))

25/01/13


In [ ]:
datetime.strftime

In [99]:
def check_fraudeur_date(date) :
    path_df = "/Users/youssouf/Downloads/simbox_v4_prediction_dt_" + date + ".parquet"
    df = spark.read.parquet(path_df)
    
    
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/simbox_fra_2809_0610.csv")
    df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN")))

    d = datetime.strptime(date , '%Y%m%d')
    d = datetime.strftime(d, "%d/%m/%y")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("Date_DETECTION") == d)
    
    res =  {"NOMBRE DE DETECTION" : df.filter(col("prediction") == 1).count() , "NOMBRE DE FRA" : df_fraudeur_FRA.count() ,
    "NOMBRE COMMUN" : df.filter(col("prediction") == 1).join(df_fraudeur, ["msisdn"], "inner").count() }

    return res

In [104]:
check_fraudeur_date("20210930")

{'NOMBRE DE DETECTION': 759573, 'NOMBRE DE FRA': 2872, 'NOMBRE COMMUN': 0}

#### Jrnée du 30/09/2021

In [110]:
df_30 = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20210930.parquet")


In [118]:
df_30.select("msisdn", "probability", "probability0", "probability1", "prediction").show(5)

+-------------+--------------------+------------------+--------------------+----------+
|       msisdn|         probability|      probability0|        probability1|prediction|
+-------------+--------------------+------------------+--------------------+----------+
|2250700000945|[0.97680815152841...|0.9768081515284116|0.023191848471588422|       0.0|
|2250700003229|[0.97680815152841...|0.9768081515284116|0.023191848471588422|       0.0|
|2250700003710|[0.08443540183112...|0.0844354018311292|  0.9155645981688708|       1.0|
|2250700004542|[0.97680815152841...|0.9768081515284116|0.023191848471588422|       0.0|
|2250700006182|[0.23142144638403...|0.2314214463840399|  0.7685785536159601|       1.0|
+-------------+--------------------+------------------+--------------------+----------+
only showing top 5 rows



In [119]:
#df_30.show(5)

In [120]:
df_30.groupby("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|2196831|
|       1.0| 793017|
+----------+-------+



In [136]:
df_30.count()

2989848

In [ ]:
df_30.filter(col("prediction") == 1)

In [121]:
df_fraudeur_FRA = spark.read.option("header", True) \
                    .option("delimiter", ";") \
                    .csv("/Users/youssouf/Downloads/simbox_fra_2809_0610.csv")
df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN")))

In [122]:
df_fr_30 = df_fraudeur_FRA.filter(col("Date_DETECTION") == "30/09/2021")


In [130]:
df_30.join(df_fr_30 , ["msisdn"], "inner").count()

90

In [131]:
df_30.filter(col("prediction") == 1).join(df_fr_30 , ["msisdn"], "inner").count()



41

In [125]:
df_final = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Downloads/trafic_global_20210930.csv")

In [126]:
df_final.join(df_fr_30 , ["msisdn"], "inner").count()



267

In [128]:
df_fr_30.count()

334

In [129]:
df_fr_30.select("msisdn").distinct().count()

334

In [134]:
df_final = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Downloads/trafic_global_20210930.csv")

df_vbm = spark.read\
    .option("header", True)\
    .option("delimiter",";")\
    .csv("/Users/youssouf/Downloads/fraudeur_simbox_vbm_all.csv")

#df_vbm.show()
df_vbm = df_vbm.withColumnRenamed("MSISDN", "MSISDN1")


df_vbm.join(df_final, df_vbm.MSISDN1 == df_final.msisdn, "right_outer").count()

7541590

In [135]:
df_vbm.join(df_final, df_vbm.MSISDN1 == df_final.msisdn, "inner").count()

7307738

In [137]:
df_30_py = spark.read.parquet("/Users/youssouf/Documents/simboxv2/prediction_dt_vbm_30_v4_dist.parquet")
df_30_py.show(5)

+-------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+----------+-------------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+-----------------+----------------+--------------------+----------------+--------------------+----------+
|msisdn1|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|date_appel|       msisdn|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeurIndex|distributeur_vec|            features|   rawPrediction|         

In [138]:
df_30_py.count()

7541590

In [139]:
df_30_py.groupby("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|4868953|
|       1.0|2672637|
+----------+-------+



In [142]:
df_30_py = df_30_py.withColumn("probability_split", vector_to_array("probability"))
df_30_py = df_30_py.select([col("probability_split")[i] for i in range(2)] + df_30_py.columns)


#df_30_py.filter(col("prediction") == 1).join(df_fr_30 , ["msisdn"], "inner").groupby("probability_split[1]").count().show()






+--------------------+-----+
|probability_split[1]|count|
+--------------------+-----+
|  0.9359783224136536|  141|
|  0.7169117647058824|    2|
|  0.9593908629441624|    5|
|                0.98|    3|
|  0.5171232876712328|    1|
|  0.7685785536159601|  103|
+--------------------+-----+



In [150]:
df_30_spark = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20210930_.parquet")
df_30_spark.show(5)


+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-----------------+----------------+--------------------+---------------+-------------------+---------------+--------------------+----------+------------------+--------------------+-----------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distr

In [151]:
df_30_spark.count()

7701561

In [143]:
import pandas as pd

In [144]:
lc = pd.read_json("/Users/youssouf/Downloads/Result_5.json")
lc.head()

,CALLING_NBR,CALLED_NBR,START_TIME,DURATION,CDR_TYPE_LV,CALLING_PREFIX_ID,CALLED_PREFIX_ID,EVENT_SEQ,RE_ID,BILLING_CYCLE_ID,...,SRC_CDR_FILE_NAME,SRC_CDR_FILE_DATE,SYS_SOURCE_FLOW,SYS_CREATE_USER,SYS_CREATE_TIMESTAMP,RE_NAME,RE_REPORT_NAME,RE_CATEGORY_LV,SYS_UPDATE_USER,SYS_UPDATE_TIMESTAMP
0,2250707292667,7.080855e+08,2021-02-06 11:29:53,0,-1,160,247,3168497210,2152,341,...,46559202,2021-02-06 12:38:26,Referentiel,OCIT,2021-04-01 19:56:44.000000,SMS_Send_Roaming_Z3_OZE,SMS_Send_Roaming_Z3_OZE,SMS ROAMING MO,init,2021-04-01 19:56:44.000000
1,2250707292667,1.707071e+11,2021-07-24 10:19:47,4,-1,160,246,1517622739,993,346,...,139998349,2021-07-24 11:39:26,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
2,2250707292667,1.707071e+11,2021-08-19 09:23:14,6,-1,160,246,3013753102,993,347,...,141605199,2021-08-19 10:16:06,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
3,2250707292667,1.707071e+11,2021-07-09 12:16:13,1,-1,160,246,2858010795,993,346,...,139087548,2021-07-09 13:12:49,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
4,2250707292667,1.707071e+11,2021-08-16 16:46:34,6,-1,160,246,4228465927,993,347,...,141446105,2021-08-16 17:46:54,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000


In [146]:
lc.dtypes

CALLING_NBR                      int64
CALLED_NBR                     float64
START_TIME              datetime64[ns]
DURATION                         int64
CDR_TYPE_LV                      int64
                             ...      
RE_NAME                         object
RE_REPORT_NAME                  object
RE_CATEGORY_LV                  object
SYS_UPDATE_USER                 object
SYS_UPDATE_TIMESTAMP            object
Length: 113, dtype: object

In [147]:
lc["CALLED_NBR"] = lc["CALLED_NBR"].apply(str)

In [148]:
lc.head()

,CALLING_NBR,CALLED_NBR,START_TIME,DURATION,CDR_TYPE_LV,CALLING_PREFIX_ID,CALLED_PREFIX_ID,EVENT_SEQ,RE_ID,BILLING_CYCLE_ID,...,SRC_CDR_FILE_NAME,SRC_CDR_FILE_DATE,SYS_SOURCE_FLOW,SYS_CREATE_USER,SYS_CREATE_TIMESTAMP,RE_NAME,RE_REPORT_NAME,RE_CATEGORY_LV,SYS_UPDATE_USER,SYS_UPDATE_TIMESTAMP
0,2250707292667,708085465.0,2021-02-06 11:29:53,0,-1,160,247,3168497210,2152,341,...,46559202,2021-02-06 12:38:26,Referentiel,OCIT,2021-04-01 19:56:44.000000,SMS_Send_Roaming_Z3_OZE,SMS_Send_Roaming_Z3_OZE,SMS ROAMING MO,init,2021-04-01 19:56:44.000000
1,2250707292667,170707070707.0,2021-07-24 10:19:47,4,-1,160,246,1517622739,993,346,...,139998349,2021-07-24 11:39:26,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
2,2250707292667,170707070707.0,2021-08-19 09:23:14,6,-1,160,246,3013753102,993,347,...,141605199,2021-08-19 10:16:06,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
3,2250707292667,170707070707.0,2021-07-09 12:16:13,1,-1,160,246,2858010795,993,346,...,139087548,2021-07-09 13:12:49,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000
4,2250707292667,170707070707.0,2021-08-16 16:46:34,6,-1,160,246,4228465927,993,347,...,141446105,2021-08-16 17:46:54,Referentiel,OCIT,2021-04-01 19:56:44.000000,Short Code Call,Short Code Call,SVA VOICE,init,2021-04-01 19:56:44.000000


In [149]:
lc.to_csv("/Users/youssouf/Downloads/result5.csv" , index = False, sep = ";" )

#### Jrnée du 05

In [152]:
df_05 = spark.read.parquet("/Users/youssouf/Downloads/simbox_v4_prediction_dt_20211005_.parquet")
df_05.show(5)


+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-----------------+----------------+--------------------+---------------+-------------------+---------------+--------------------+----------+------------------+--------------------+-----------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distr

In [ ]:
#df_fr_30 = df_fraudeur_FRA.filter(col("Date_DETECTION") == "30/09/2021")


In [153]:
df_fr_05 = df_fraudeur_FRA.filter(col("Date_DETECTION") == "05/10/2021")
df_fr_05.show(5)


+--------------+-------------+
|Date_DETECTION|       msisdn|
+--------------+-------------+
|    05/10/2021|2250700022928|
|    05/10/2021|2250700064483|
|    05/10/2021|2250700064923|
|    05/10/2021|2250700308527|
|    05/10/2021|2250700330973|
+--------------+-------------+
only showing top 5 rows



In [154]:
df_05.filter(col("prediction") == 1).join(df_fr_05 , ["msisdn"], "inner").count()


152

In [155]:
df_fr_05.count()

348

In [161]:
#df_05 = df_05.withColumn("probability_split", vector_to_array("probability"))
#df_05 = df_05.select([col("probability_split")[i] for i in range(2)] + df_05.columns)

df_05.filter(col("prediction") == 1).join(df_fr_05 , ["msisdn"], "inner").groupby("probability1").count().show()



+------------------+-----+
|      probability1|count|
+------------------+-----+
|0.9155645981688708|  114|
|0.7685785536159601|   38|
+------------------+-----+



In [188]:
from sklearn import tree


In [164]:
ff = pd.read_csv("/Users/youssouf/Documents/simboxv2/simbox_v4_3.csv", sep = ";")
ff.head()


,msisdn,distributeur,statut_voix,statut_sms,statut_ff,statut_sva,statut_data,statut_om,valeur,vol_tot_data,...,nbre_dist_sms_in,nbre_voix_in,dur_in,nbre_dist_voix_in,nbre_sms_night,nbre_dist_sms_night,nbre_voix_night,dur_night,nbre_dist_voix_night,statut
0,2250707492509,AMANDA,1.0,1.0,1.0,1.0,1.0,1.0,8249.0,1.0,...,1,0,0,0,0,0,0,0,0,0
1,2250708078700,AMANDA,1.0,1.0,1.0,1.0,1.0,1.0,8249.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2250708160010,AMANDA,1.0,1.0,1.0,1.0,1.0,1.0,8249.0,1.0,...,1,0,0,0,0,0,0,0,0,0
3,2250708643787,AMANDA,1.0,1.0,1.0,1.0,1.0,1.0,8249.0,1.0,...,1,0,0,0,0,0,0,0,0,0
4,2250747888117,AMANDA,1.0,1.0,1.0,1.0,1.0,1.0,8249.0,1.0,...,1,0,0,0,0,0,0,0,0,0


In [166]:
ff.dtypes


msisdn                    int64
distributeur             object
statut_voix             float64
statut_sms              float64
statut_ff               float64
statut_sva              float64
statut_data             float64
statut_om               float64
valeur                  float64
vol_tot_data            float64
vol_tot_voix_offnet     float64
vol_tot_voix_onet       float64
date_appel               object
nbre_sms_out              int64
nbre_dist_sms_out         int64
nbre_voix_out             int64
dur_out                   int64
nbre_dist_voix_out        int64
nbre_cell_out             int64
nbre_sms_in               int64
nbre_dist_sms_in          int64
nbre_voix_in              int64
dur_in                    int64
nbre_dist_voix_in         int64
nbre_sms_night            int64
nbre_dist_sms_night       int64
nbre_voix_night           int64
dur_night                 int64
nbre_dist_voix_night      int64
statut                    int64
dtype: object

In [167]:
ff.columns

Index(['msisdn', 'distributeur', 'statut_voix', 'statut_sms', 'statut_ff',
       'statut_sva', 'statut_data', 'statut_om', 'valeur', 'vol_tot_data',
       'vol_tot_voix_offnet', 'vol_tot_voix_onet', 'date_appel',
       'nbre_sms_out', 'nbre_dist_sms_out', 'nbre_voix_out', 'dur_out',
       'nbre_dist_voix_out', 'nbre_cell_out', 'nbre_sms_in',
       'nbre_dist_sms_in', 'nbre_voix_in', 'dur_in', 'nbre_dist_voix_in',
       'nbre_sms_night', 'nbre_dist_sms_night', 'nbre_voix_night', 'dur_night',
       'nbre_dist_voix_night', 'statut'],
      dtype='object')

In [172]:
data = ff[['statut_voix', 'statut_sms', 'statut_ff',
       'statut_sva', 'statut_data', 'statut_om', 'valeur', 'vol_tot_data',
       'vol_tot_voix_offnet', 'vol_tot_voix_onet',
       'nbre_sms_out', 'nbre_dist_sms_out', 'nbre_voix_out', 'dur_out',
       'nbre_dist_voix_out', 'nbre_cell_out', 'nbre_sms_in',
       'nbre_dist_sms_in', 'nbre_voix_in', 'dur_in', 'nbre_dist_voix_in',
       'nbre_sms_night', 'nbre_dist_sms_night', 'nbre_voix_night', 'dur_night',
       'nbre_dist_voix_night']]

target = ff[['statut']]

In [175]:
data = data.fillna(0)

In [176]:
from sklearn.datasets import load_iris
from sklearn import tree

# Load in our dataset
#iris_data = load_iris()

# Initialize our decision tree object
classification_tree = tree.DecisionTreeClassifier()

# Train our decision tree (tree induction + pruning)
classification_tree = classification_tree.fit(data , target)

In [184]:
classification_tree.feature_importances_

array([2.16946585e-02, 8.48027720e-03, 7.67409337e-03, 3.07303132e-02,
       1.76173435e-02, 3.85204555e-02, 2.97456430e-01, 6.63828948e-02,
       4.28430089e-02, 8.63418534e-02, 4.39432860e-02, 4.78044650e-02,
       4.05676953e-02, 1.02236399e-01, 5.70372825e-02, 4.20047339e-02,
       5.74002497e-03, 5.25569095e-03, 2.74653909e-02, 1.50522194e-03,
       7.23305057e-04, 3.80811452e-03, 2.23022359e-03, 1.72043807e-04,
       1.01989225e-03, 7.44901920e-04])

In [189]:
text_representation = tree.export_text(classification_tree)
print(text_representation)

|--- feature_9 <= 0.50
|   |--- feature_6 <= 4999.00
|   |   |--- feature_18 <= 1.50
|   |   |   |--- feature_11 <= 1.50
|   |   |   |   |--- feature_10 <= 0.50
|   |   |   |   |   |--- feature_17 <= 1.50
|   |   |   |   |   |   |--- feature_22 <= 1.50
|   |   |   |   |   |   |   |--- feature_19 <= 5.50
|   |   |   |   |   |   |   |   |--- feature_5 <= 0.50
|   |   |   |   |   |   |   |   |   |--- feature_16 <= 2.50
|   |   |   |   |   |   |   |   |   |   |--- feature_22 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 13
|   |   |   |   |   |   |   |   |   |   |--- feature_22 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 22
|   |   |   |   |   |   |   |   |   |--- feature_16 >  2.50
|   |   |   |   |   |   |   |   |   |   |--- feature_6 <= 50.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 7
|   |   |   |   |   |   |   |   |   |   |--- feature_6 >  50.00
|   |   |   |   |   |   |   |  

In [190]:
data.columns

Index(['statut_voix', 'statut_sms', 'statut_ff', 'statut_sva', 'statut_data',
       'statut_om', 'valeur', 'vol_tot_data', 'vol_tot_voix_offnet',
       'vol_tot_voix_onet', 'nbre_sms_out', 'nbre_dist_sms_out',
       'nbre_voix_out', 'dur_out', 'nbre_dist_voix_out', 'nbre_cell_out',
       'nbre_sms_in', 'nbre_dist_sms_in', 'nbre_voix_in', 'dur_in',
       'nbre_dist_voix_in', 'nbre_sms_night', 'nbre_dist_sms_night',
       'nbre_voix_night', 'dur_night', 'nbre_dist_voix_night'],
      dtype='object')

In [197]:
#dot_data = tree.export_graphviz(classification_tree, out_file=None, 
#                     feature_names=data.columns,  
#                     class_names= [0,1] ,  
#                     filled=True, rounded=True,  
#                     special_characters=True)  
#graph = graphviz.Source(dot_data)  

In [191]:
import graphviz 

#graph.render("iris") 

In [195]:
from dtreeviz.trees import dtreeviz # remember to load the package

#viz = dtreeviz(classification_tree , data , target,
#               target_name="target",
#                feature_names=data.columns ,
#                class_names=[0,1])

#viz

##### Pyspark Decision Tree

In [ ]:
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={}, impurity='variance', numClasses=2)

paramGrid = ParamGridBuilder() \
    .addGrid(model.maxDepth, [4, 5, 6, 7]) \
    .addGrid(model.maxBins, [24, 28, 32, 36]) \
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)  

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

In [198]:
df = spark.read.\
       option("header","true").\
       option("delimiter",";").\
       csv("/Users/youssouf/Documents/simboxv2/simbox_v4_3.csv")

In [202]:
#df.toPandas()

In [ ]:
tt.head()

#### Kmeans with transformed variables

In [206]:
df_transf = spark.read.option("header", True).option("delimiter", ";").csv("/Users/youssouf/Documents/simboxv2/transformed_var.csv")
df_transf.show(5)



+------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|statut|            valeur|       vol_tot_data|vol_tot_voix_offnet| vol_tot_voix_onet|       nbre_sms_out|  nbre_dist_sms_out|      nbre_voix_out|            dur_out| nbre_dist_voix_out|      nbre_cell_out|       nbre_sms_in|   nbre_dist_sms_in|        nbre_voix_in|              dur_in|   nbre_dist_voix_in|    nbre_sms_night| nbre_dist_sms_night|     nbre_voix_night|           dur_night|nbre_dist_voix_night|
+------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+----------

In [207]:
tranf_col_selected = ["valeur"  ,             "vol_tot_data"  ,       "vol_tot_voix_offnet",  "vol_tot_voix_onet"   ,
"nbre_sms_out"   ,      "nbre_dist_sms_out"  ,  "nbre_voix_out"   ,     "dur_out"    ,         
"nbre_dist_voix_out" ,  "nbre_cell_out"  ,      "nbre_sms_in"    ,      "nbre_dist_sms_in"   , 
"nbre_voix_in"  ,       "dur_in"     ,          "nbre_dist_voix_in" ,   "nbre_sms_night"    ,  
"nbre_dist_sms_night" , "nbre_voix_night"   ,   "dur_night"    ,        "nbre_dist_voix_night"]

In [211]:
for cl in tranf_col_selected:
    df_transf = df_transf.withColumn(cl, col(cl).cast(DoubleType()))

In [212]:
df_transf1 = df_transf.select(tranf_col_selected)

In [216]:
df_transf1.printSchema()

root
 |-- valeur: double (nullable = true)
 |-- vol_tot_data: double (nullable = true)
 |-- vol_tot_voix_offnet: double (nullable = true)
 |-- vol_tot_voix_onet: double (nullable = true)
 |-- nbre_sms_out: double (nullable = true)
 |-- nbre_dist_sms_out: double (nullable = true)
 |-- nbre_voix_out: double (nullable = true)
 |-- dur_out: double (nullable = true)
 |-- nbre_dist_voix_out: double (nullable = true)
 |-- nbre_cell_out: double (nullable = true)
 |-- nbre_sms_in: double (nullable = true)
 |-- nbre_dist_sms_in: double (nullable = true)
 |-- nbre_voix_in: double (nullable = true)
 |-- dur_in: double (nullable = true)
 |-- nbre_dist_voix_in: double (nullable = true)
 |-- nbre_sms_night: double (nullable = true)
 |-- nbre_dist_sms_night: double (nullable = true)
 |-- nbre_voix_night: double (nullable = true)
 |-- dur_night: double (nullable = true)
 |-- nbre_dist_voix_night: double (nullable = true)



In [222]:
assemble=VectorAssembler(inputCols=df_transf1.columns, outputCol='features').setHandleInvalid("keep")
assembled_data=assemble.transform(df_transf1)

scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)



In [225]:
# Trains a k-means model.
kmeans = KMeans().setK(3).setSeed(123)
model = kmeans.fit(assembled_data)

# Make predictions
predictions = model.transform(data_scale_output)

Py4JJavaError: An error occurred while calling o2880.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 425.0 failed 1 times, most recent failure: Lost task 2.0 in stage 425.0 (TID 6457) (apache-guacamole.dev01.smile.lan executor driver): java.io.EOFException: Cannot seek after EOF
	at org.apache.hadoop.fs.ChecksumFileSystem$FSDataBoundedInputStream.seek(ChecksumFileSystem.java:329)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.initialize(LineRecordReader.java:115)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.<init>(HadoopFileLinesReader.scala:65)
	at org.apache.spark.sql.execution.datasources.csv.TextInputCSVDataSource$.readFile(CSVDataSource.scala:97)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat.$anonfun$buildReader$1(CSVFileFormat.scala:148)
	at org.apache.spark.sql.execution.datasources.FileFormat$$anon$1.apply(FileFormat.scala:147)
	at org.apache.spark.sql.execution.datasources.FileForm

In [ ]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

##### Yeo Johnson Function

In [227]:
def yeojohnson(x, lmbda) : 
    if (x >= 0, lmbda != 0) :
        return ((x + 1)**lmbda - 1) / lmbda
    elif (x >= 0, lmbda == 0) :
        return log(x + 1)
    elif (x < 0, lmbda != 2) :
        return -((-x + 1)**(2 - lmbda) - 1) / (2 - lmbda)
    elif (x < 0, lmbda == 2) : 
        return -log(-x + 1)

In [228]:
yeojohnson(2,3)

8.666666666666666

In [ ]:
   valeur          vol_tot_data    vol_tot_voix_offnet vol_tot_voix_onet nbre_sms_out   
  0.2362504       0.09039554      0.1051099           0.1859025         -0.7436904     
     nbre_dist_sms_out nbre_voix_out   dur_out         nbre_dist_voix_out nbre_cell_out  
 -0.9938551        -1.231552       -0.4054642      -1.499216          -1.28494       
     nbre_sms_in     nbre_dist_sms_in nbre_voix_in    dur_in          nbre_dist_voix_in
 -1.245326       -1.08071         -4.999952       -4.99994        -4.99994         
     nbre_sms_night  nbre_dist_sms_night nbre_voix_night dur_night       nbre_dist_voix_night
 -4.99994        -4.99994            -4.999959       -4.99994        -4.999943 

In [229]:
yeojohnsonUDF = udf(lambda z: yeojohnson(z)) 

In [ ]:
df_transf.select("valeur","vol_tot_data","vol_tot_voix_offnet","vol_tot_voix_onet","nbre_sms_out").show(5)